<a href="https://colab.research.google.com/github/trangnguyenvn1398/loan-recommendation/blob/main/loan-rec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import packages

In [1]:
# import packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegressionCV, LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelBinarizer
from sklearn.metrics import accuracy_score, r2_score, mean_absolute_error, mean_squared_error, median_absolute_error, precision_recall_fscore_support
from sklearn.utils import shuffle

# Load and preprocess data

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [3]:
# load the data

rejected = pd.read_csv('/content/gdrive/My Drive/Fall 2020/CS156/rejected_2007_to_2018Q4.csv')
accepted = pd.read_csv('/content/gdrive/My Drive/Fall 2020/CS156/accepted_2007_to_2018Q4.csv')

# check the data
print(f'Rejected shape: {rejected.shape}, Accepted shape: {accepted.shape}')
print('Rejected head', rejected.head)
print('Accepted head', accepted.head)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Rejected shape: (27648741, 9), Accepted shape: (2260701, 151)
Rejected head <bound method NDFrame.head of           Amount Requested Application Date  ... Employment Length  Policy Code
0                   1000.0       2007-05-26  ...           4 years          0.0
1                   1000.0       2007-05-26  ...          < 1 year          0.0
2                  11000.0       2007-05-27  ...            1 year          0.0
3                   6000.0       2007-05-27  ...          < 1 year          0.0
4                   1500.0       2007-05-27  ...          < 1 year          0.0
...                    ...              ...  ...               ...          ...
27648736           10000.0       2016-12-31  ...          < 1 year          0.0
27648737           10000.0       2016-12-31  ...           5 years          0.0
27648738            1200.0       2016-12-31  ...          < 1 year          0.0
27648739           25000.0       2016-12-31  ...          < 1 year          0.0
27648740      

In [4]:
# check the columns
rejected_cols = rejected.columns
accepted_cols = accepted.columns
print('Rejected columns', rejected_cols)
print('Accepted columns', accepted_cols)

Rejected columns Index(['Amount Requested', 'Application Date', 'Loan Title', 'Risk_Score',
       'Debt-To-Income Ratio', 'Zip Code', 'State', 'Employment Length',
       'Policy Code'],
      dtype='object')
Accepted columns Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       ...
       'hardship_payoff_balance_amount', 'hardship_last_payment_amount',
       'disbursement_method', 'debt_settlement_flag',
       'debt_settlement_flag_date', 'settlement_status', 'settlement_date',
       'settlement_amount', 'settlement_percentage', 'settlement_term'],
      dtype='object', length=151)


In [5]:
# get sample from the data

sampled_rejected = rejected.sample(n=3000, random_state=123)
sampled_accepted = accepted.sample(n=10000, random_state=123)

sampled_rejected = sampled_rejected.drop(['Application Date', 'Risk_Score'], axis=1)
sampled_accepted = sampled_accepted[['funded_amnt', 'funded_amnt_inv', 'loan_amnt', 'loan_status', 'title', 'dti', 'zip_code', 'addr_state', 'emp_length', 'policy_code']]

In [6]:
# convert to a csv file
sampled_rejected.to_csv('sample_rejected.csv')
sampled_accepted.to_csv('sample_accepted.csv')

In [7]:
# check the accepted sample
sampled_accepted.head()

,funded_amnt,funded_amnt_inv,loan_amnt,loan_status,title,dti,zip_code,addr_state,emp_length,policy_code
694363,21550.0,21550.0,21550.0,Charged Off,Debt consolidation,14.02,894xx,NV,< 1 year,1.0
471099,10500.0,10500.0,10500.0,Fully Paid,Debt consolidation,20.67,985xx,WA,10+ years,1.0
497651,10000.0,10000.0,10000.0,Current,Debt consolidation,28.70,937xx,CA,10+ years,1.0
1424167,27000.0,27000.0,27000.0,Current,Debt consolidation,17.46,178xx,PA,2 years,1.0
1886646,35000.0,35000.0,35000.0,Fully Paid,Investments,6.66,021xx,MA,10+ years,1.0


In [8]:
sampled_accepted[sampled_accepted['funded_amnt'] != sampled_accepted['loan_amnt']]

,funded_amnt,funded_amnt_inv,loan_amnt,loan_status,title,dti,zip_code,addr_state,emp_length,policy_code
1623446,10575.0,10575.000000,13700.0,Charged Off,Impact Fee,0.00,082xx,NJ,6 years,1.0
1625126,11000.0,10750.000000,16000.0,Fully Paid,Debt Consolidation,16.75,917xx,CA,9 years,1.0
1625481,24600.0,24575.000000,28000.0,Fully Paid,Home improvement,7.14,922xx,CA,10+ years,1.0
1624257,9175.0,9150.000000,10000.0,Fully Paid,2011 Pay Down Dept,15.07,662xx,KS,10+ years,1.0
1627127,21275.0,21225.000000,35000.0,Charged Off,Business Loan,6.48,597xx,MT,8 years,1.0
1640915,9600.0,9215.624001,14400.0,Fully Paid,Wedding and Debt,21.60,949xx,CA,< 1 year,1.0
1636518,15700.0,15456.031031,25000.0,Charged Off,House Projects,16.26,928xx,CA,10+ years,1.0
1634685,6300.0,5887.864277,10000.0,Fully Paid,Credit Card Refinance,8.64,921xx,CA,5 years,1.0
1612960,16175.0,16175.000000,17475.0,Charged Off,personal loan,24.84,890xx,NV,10+ years,1.0
1620170,12050.0,12046.260282,16000.0,Fully Paid,Credit Card Payoff,16.63,920xx,CA,< 1 year,1.0


In [9]:
sampled_accepted['accepted'] = 1
sampled_accepted = sampled_accepted[['title',
       'dti', 'zip_code', 'addr_state', 'emp_length', 'policy_code', 'loan_amnt', 'accepted', 'funded_amnt', 'funded_amnt_inv']]

In [10]:
sampled_rejected.head()

,Amount Requested,Loan Title,Debt-To-Income Ratio,Zip Code,State,Employment Length,Policy Code
6160574,20000.0,Credit card refinancing,100%,452xx,OH,< 1 year,0.0
4100285,25000.0,debt_consolidation,1.14%,948xx,CA,5 years,0.0
188890,25000.0,other,19.71%,335xx,FL,< 1 year,0.0
8575099,8000.0,moving,14.69%,770xx,TX,< 1 year,0.0
4220478,5000.0,Debt consolidation,11.19%,299xx,SC,< 1 year,0.0


In [11]:
sampled_rejected['funded_amnt'] = 0
sampled_rejected['funded_amnt_inv'] = 0
sampled_rejected['accepted'] = 0

sampled_rejected = sampled_rejected[['Loan Title', 'Debt-To-Income Ratio', 'Zip Code',
       'State', 'Employment Length', 'Policy Code', 'Amount Requested', 'accepted', 'funded_amnt',
       'funded_amnt_inv']]
sampled_rejected.columns = sampled_accepted.columns

In [12]:
sampled_accepted.head()

,title,dti,zip_code,addr_state,emp_length,policy_code,loan_amnt,accepted,funded_amnt,funded_amnt_inv
694363,Debt consolidation,14.02,894xx,NV,< 1 year,1.0,21550.0,1,21550.0,21550.0
471099,Debt consolidation,20.67,985xx,WA,10+ years,1.0,10500.0,1,10500.0,10500.0
497651,Debt consolidation,28.70,937xx,CA,10+ years,1.0,10000.0,1,10000.0,10000.0
1424167,Debt consolidation,17.46,178xx,PA,2 years,1.0,27000.0,1,27000.0,27000.0
1886646,Investments,6.66,021xx,MA,10+ years,1.0,35000.0,1,35000.0,35000.0


In [13]:
# check the rejected sample
sampled_rejected.head()

,title,dti,zip_code,addr_state,emp_length,policy_code,loan_amnt,accepted,funded_amnt,funded_amnt_inv
6160574,Credit card refinancing,100%,452xx,OH,< 1 year,0.0,20000.0,0,0,0
4100285,debt_consolidation,1.14%,948xx,CA,5 years,0.0,25000.0,0,0,0
188890,other,19.71%,335xx,FL,< 1 year,0.0,25000.0,0,0,0
8575099,moving,14.69%,770xx,TX,< 1 year,0.0,8000.0,0,0,0
4220478,Debt consolidation,11.19%,299xx,SC,< 1 year,0.0,5000.0,0,0,0


In [14]:
# join accepted and rejected, shuffle
sampled = pd.concat([sampled_accepted, sampled_rejected])
sampled = shuffle(sampled).reset_index(drop=True)

In [15]:
sampled.head(10)

,title,dti,zip_code,addr_state,emp_length,policy_code,loan_amnt,accepted,funded_amnt,funded_amnt_inv
0,Debt consolidation,33.06%,220xx,VA,< 1 year,0.0,40000.0,0,0.0,0.0
1,Home improvement,53.71,810xx,CO,10+ years,1.0,10000.0,1,10000.0,10000.0
2,Other,7.09%,606xx,IL,< 1 year,0.0,4000.0,0,0.0,0.0
3,other,51.63%,170xx,PA,< 1 year,0.0,15000.0,0,0.0,0.0
4,Debt consolidation,29.98,114xx,NY,1 year,1.0,20000.0,1,20000.0,20000.0
5,Debt consolidation,7.84,925xx,CA,2 years,1.0,10100.0,1,10100.0,10100.0
6,other,0%,104xx,NY,5 years,0.0,6000.0,0,0.0,0.0
7,Debt consolidation,27.6,974xx,OR,10+ years,1.0,8000.0,1,8000.0,8000.0
8,Credit card refinancing,13.64,120xx,NY,< 1 year,1.0,28000.0,1,28000.0,28000.0
9,Debt consolidation,16.67,774xx,TX,10+ years,1.0,11000.0,1,11000.0,11000.0


In [16]:
sampled['emp_length'].unique()

array(['< 1 year', '10+ years', '1 year', '2 years', '5 years', nan,
       '4 years', '6 years', '3 years', '9 years', '8 years', '7 years'],
      dtype=object)

In [17]:
# convert employment years to numbers
def emp_year(data):
  if data == '< 1 year':
    return 0
  elif data == '10+ years':
    return 10
  elif 'years' in str(data) or 'year' in str(data):
    return np.float(data[0])
  else:
    return data

sampled['emp_length'] = sampled['emp_length'].apply(emp_year)

In [18]:
# convert dti to percentages
def dti_short(data):
  if '%' in str(data):
    return np.float(str(data)[:-1])
  else:
    return np.float(data)

sampled['dti'] = sampled['dti'].apply(dti_short)

In [19]:
# get the first number of zip code
sampled['zip_code_short'] = sampled['zip_code'].str[:1]
sampled['zip_code_short'].unique()

array(['2', '8', '6', '1', '9', '7', '0', '3', '4', '5'], dtype=object)

In [20]:
# check the states
sampled['addr_state'].unique()

array(['VA', 'CO', 'IL', 'PA', 'NY', 'CA', 'OR', 'TX', 'NJ', 'GA', 'OH',
       'ID', 'WA', 'NC', 'FL', 'AZ', 'MI', 'WI', 'MD', 'TN', 'MA', 'KS',
       'LA', 'MO', 'SC', 'RI', 'IN', 'NV', 'MS', 'ME', 'MN', 'HI', 'OK',
       'AK', 'AR', 'UT', 'DE', 'DC', 'KY', 'WV', 'VT', 'AL', 'NH', 'CT',
       'NM', 'WY', 'NE', 'SD', 'MT', 'ND'], dtype=object)

In [21]:
# check the policy code
sampled['policy_code'][sampled['accepted']==0].value_counts()

0.0    2989
2.0      11
Name: policy_code, dtype: int64

In [22]:
sampled['policy_code'][sampled['accepted']==1].value_counts()

1.0    10000
Name: policy_code, dtype: int64

In [23]:
sampled.head()

,title,dti,zip_code,addr_state,emp_length,policy_code,loan_amnt,accepted,funded_amnt,funded_amnt_inv,zip_code_short
0,Debt consolidation,33.06,220xx,VA,0.0,0.0,40000.0,0,0.0,0.0,2
1,Home improvement,53.71,810xx,CO,10.0,1.0,10000.0,1,10000.0,10000.0,8
2,Other,7.09,606xx,IL,0.0,0.0,4000.0,0,0.0,0.0,6
3,other,51.63,170xx,PA,0.0,0.0,15000.0,0,0.0,0.0,1
4,Debt consolidation,29.98,114xx,NY,1.0,1.0,20000.0,1,20000.0,20000.0,1


In [24]:
# getting training and test set
X_train, X_test, y_train, y_test = train_test_split(sampled.drop(['accepted', 'funded_amnt', 'funded_amnt_inv'], axis=1),\
                                                                     sampled[['accepted', 'funded_amnt', 'funded_amnt_inv']], test_size=0.3, random_state=123)

In [25]:
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [26]:
# binarize the important categorical data

lb1 = LabelBinarizer()
lb2 = LabelBinarizer()

numerical_train = X_train.select_dtypes(include=['number']).drop(['policy_code'], axis=1)
X_train = pd.concat([numerical_train, pd.DataFrame(lb1.fit_transform(X_train['zip_code_short'].to_numpy().reshape(-1,1))), pd.DataFrame(lb2.fit_transform(X_train['policy_code'].to_numpy().reshape(-1,1)))], axis=1)

numerical_test = X_test.select_dtypes(include=['number']).drop(['policy_code'], axis=1)
X_test = pd.concat([numerical_test, pd.DataFrame(lb1.transform(X_test['zip_code_short'].to_numpy().reshape(-1,1))), pd.DataFrame(lb2.transform(X_test['policy_code'].to_numpy().reshape(-1,1)))], axis=1)

In [27]:
# check na values
print(X_train.shape)
X_train.isna().sum()

(9100, 16)


dti             4
emp_length    525
loan_amnt       0
0               0
1               0
2               0
3               0
4               0
5               0
6               0
7               0
8               0
9               0
0               0
1               0
2               0
dtype: int64

In [28]:
# drop na values
y_train = y_train.iloc[X_train.dropna(subset=['dti', 'emp_length']).index].reset_index(drop=True)
X_train = X_train.dropna(subset=['dti', 'emp_length']).reset_index(drop=True)

In [29]:
print(X_train.shape)
X_train.isna().sum()

(8575, 16)


dti           0
emp_length    0
loan_amnt     0
0             0
1             0
2             0
3             0
4             0
5             0
6             0
7             0
8             0
9             0
0             0
1             0
2             0
dtype: int64

In [30]:
X_train.head()

,dti,emp_length,loan_amnt,0,1,2,3,4,5,6,7,8,9,0,1,2
0,21.71,8.0,14700.0,0,0,0,1,0,0,0,0,0,0,0,1,0
1,19.25,2.0,5000.0,1,0,0,0,0,0,0,0,0,0,0,1,0
2,20.72,5.0,35000.0,0,0,0,0,1,0,0,0,0,0,1,0,0
3,19.47,3.0,10000.0,0,1,0,0,0,0,0,0,0,0,1,0,0
4,23.64,6.0,4075.0,0,0,0,0,0,0,0,0,0,1,0,1,0


In [31]:
# normalize the non-binary data
ct = ColumnTransformer([
        ('train_scaled', StandardScaler(), ['dti', 'emp_length', 'loan_amnt'])
    ], remainder='drop')

X_train[['dti', 'emp_length', 'loan_amnt']] = pd.DataFrame(ct.fit_transform(X_train))

In [32]:
X_train.head()

,dti,emp_length,loan_amnt,0,1,2,3,4,5,6,7,8,9,0,1,2
0,-0.030165,0.823254,-0.010714,0,0,0,1,0,0,0,0,0,0,0,1,0
1,-0.036742,-0.658465,-0.860517,1,0,0,0,0,0,0,0,0,0,0,1,0
2,-0.032812,0.082394,1.767740,0,0,0,0,1,0,0,0,0,0,1,0,0
3,-0.036154,-0.411512,-0.422474,0,1,0,0,0,0,0,0,0,0,1,0,0
4,-0.025004,0.329348,-0.941555,0,0,0,0,0,0,0,0,0,1,0,1,0


In [33]:
# check na values for test set
print(X_test.shape)
X_test.isna().sum()

(3900, 16)


dti             0
emp_length    218
loan_amnt       0
0               0
1               0
2               0
3               0
4               0
5               0
6               0
7               0
8               0
9               0
0               0
1               0
2               0
dtype: int64

In [34]:
# drop na values for test set
y_test = y_test.iloc[X_test.dropna(subset=['dti', 'emp_length']).index].reset_index(drop=True)
X_test = X_test.dropna(subset=['dti', 'emp_length']).reset_index(drop=True)

In [35]:
print(X_test.shape)
X_test.isna().sum()

(3682, 16)


dti           0
emp_length    0
loan_amnt     0
0             0
1             0
2             0
3             0
4             0
5             0
6             0
7             0
8             0
9             0
0             0
1             0
2             0
dtype: int64

In [36]:
# normalize test set based on training set
X_test[['dti', 'emp_length', 'loan_amnt']] = pd.DataFrame(ct.transform(X_test))

In [37]:
X_test.head()

,dti,emp_length,loan_amnt,0,1,2,3,4,5,6,7,8,9,0,1,2
0,-0.038026,1.317160,-0.930604,1,0,0,0,0,0,0,0,0,0,0,1,0
1,-0.041395,-1.152371,-0.247257,0,0,0,0,0,0,0,1,0,0,0,1,0
2,0.179170,-1.152371,-1.035734,0,0,0,0,0,0,0,0,0,1,1,0,0
3,-0.064390,1.317160,1.329697,0,0,1,0,0,0,0,0,0,0,0,1,0
4,0.049756,-1.152371,0.015569,0,0,0,0,0,0,0,1,0,0,1,0,0


In [38]:
y_train.head()

,accepted,funded_amnt,funded_amnt_inv
0,1,14700.0,14700.0
1,1,5000.0,5000.0
2,0,0.0,0.0
3,0,0.0,0.0
4,1,4075.0,4075.0


# Build models

## Regression

### Linear Regression

In [39]:
# build and run linear regression
lr = LinearRegression()
lr.fit(X_train, y_train['funded_amnt'])
y_train_pred_funded_amnt = lr.predict(X_train)
y_test_pred_funded_amnt = lr.predict(X_test)
print('MSE for train', mean_squared_error(y_train['funded_amnt'], y_train_pred_funded_amnt))
print('MSE for test', mean_squared_error(y_test['funded_amnt'], y_test_pred_funded_amnt))
print('R^2 score for train', lr.score(X_train, y_train['funded_amnt']))
print('R^2 score for test', lr.score(X_test, y_test['funded_amnt']))

MSE for train 32150092.873799767
MSE for test 30843036.508832984
R^2 score for train 0.6964914912305782
R^2 score for test 0.7169902107575576


In [40]:
# build and run linear ridge regression
ridge = RidgeCV()
ridge.fit(X_train, y_train['funded_amnt'])
y_train_pred_funded_amnt = ridge.predict(X_train)
y_test_pred_funded_amnt = ridge.predict(X_test)
print('MSE for train', mean_squared_error(y_train['funded_amnt'], y_train_pred_funded_amnt))
print('MSE for test', mean_squared_error(y_test['funded_amnt'], y_test_pred_funded_amnt))
print('R^2 score for train', ridge.score(X_train, y_train['funded_amnt']))
print('R^2 score for test', ridge.score(X_test, y_test['funded_amnt']))

MSE for train 32152545.79953366
MSE for test 30826942.532530036
R^2 score for train 0.6964683347241711
R^2 score for test 0.717137885998297


### Random Forest Regressor

In [44]:
# build and run random forest regressor
rfr = RandomForestRegressor(random_state=123)
rfr.fit(X_train, y_train['funded_amnt'])
y_train_pred_funded_amnt = rfr.predict(X_train)
y_test_pred_funded_amnt = rfr.predict(X_test)
print('MSE for train', mean_squared_error(y_train['funded_amnt'], y_train_pred_funded_amnt))
print('MSE for test', mean_squared_error(y_test['funded_amnt'], y_test_pred_funded_amnt))
print('R^2 for train', rfr.score(X_train, y_train['funded_amnt']))
print('R^2 for test', rfr.score(X_test, y_test['funded_amnt']))

MSE for train 5107.924052478134
MSE for test 56643.11715779467
R^2 for train 0.9999517793488759
R^2 for test 0.9994802536175622


## Classification

In [45]:
# cross validation for random forest classifier

param_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}

rfc_ = RandomForestClassifier(random_state=123)
grc = GridSearchCV(rfc_, param_grid=param_grid, cv=7)
grc.fit(X_train, y_train['accepted'])
grc.best_params_

{'criterion': 'gini',
 'max_depth': 4,
 'max_features': 'auto',
 'n_estimators': 200}

In [46]:
# build and run random forest classifier
rfc = RandomForestClassifier(random_state=123, max_features='auto', n_estimators= 200, max_depth=4, criterion='gini')
rfc.fit(X_train, y_train['accepted'])
y_train_pred_status = rfc.predict(X_train)
y_test_pred_status = rfc.predict(X_test)
print('accuracy for train', accuracy_score(y_train['accepted'], y_train_pred_status))
print('accuracy for test', accuracy_score(y_test['accepted'], y_test_pred_status))
print('precision, reall, f-score, support for train', precision_recall_fscore_support(y_train['accepted'], y_train_pred_status))
print('precision, reall, f-score, support for test', precision_recall_fscore_support(y_test['accepted'], y_test_pred_status))

accuracy for train 1.0
accuracy for test 1.0
precision, reall, f-score, support for train (array([1., 1.]), array([1., 1.]), array([1., 1.]), array([2040, 6535]))
precision, reall, f-score, support for test (array([1., 1.]), array([1., 1.]), array([1., 1.]), array([ 853, 2829]))
